In [13]:
#!/usr/bin/env python
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
# Make sure that caffe is on the python path:
caffe_root = '/home/yue/caffe-rc5/'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe
import time

In [14]:
def colorize(y, ycrcb):
    y[y>255] = 255
    y[y<0] = 0

    img = np.zeros((y.shape[0], y.shape[1], 3), np.uint8)
    img[:,:,0] = y
    img[:,:,1] = ycrcb[:,:,1]
    img[:,:,2] = ycrcb[:,:,2]
    img = cv2.cvtColor(img, cv2.COLOR_YCR_CB2RGB)
    
    return img

# PSNR measure, from ANR's code
def PSNR(pred, gt):
    f = pred.astype(float)
    g = gt.astype(float)
    e = (f - g).flatten()
    n = len(e)
    rst = 10*np.log10(n/e.dot(e))
    
    return rst

In [15]:
net = caffe.Net(caffe_root + 'examples/SRCNN/SRCNN.prototxt',
                       caffe_root + 'examples/SRCNN/SRCNN_iter_11112465.caffemodel', caffe.TEST)
# Inputs
im_raw = cv2.imread('./Test/Set14/face_2.bmp')


im_resize = cv2.resize(im_raw, (586, 391))
ycrcb = cv2.cvtColor(im_resize, cv2.COLOR_RGB2YCR_CB)

im_resize = ycrcb[:,:,0]
im_resize = im_resize.reshape((im_resize.shape[0], im_resize.shape[1], 1))

# Switch input
im_input = im_resize

# plt.show()


caffe_transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
caffe_transformer.set_transpose('data', (2, 0, 1))
caffe_transformer.set_raw_scale('data', 255.0)

start_time = time.time()
out = net.forward_all(data=np.asarray([caffe_transformer.preprocess('data', im_input.astype(float)/255)]))
infer_time = (time.time() - start_time)*1000
print('infer time: ', infer_time)

# Predict results
mat = out['conv3'][0]

# Show

ycrcb = ycrcb[5:-5,5:-5,:]
im_pred = colorize(mat[0,:,:], ycrcb)
im_pred = cv2.resize(im_pred, (im_raw.shape[1],im_raw.shape[0]))

cv2.imshow('prediction',im_pred)
cv2.imshow('input',im_raw)
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/yue/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/yue/.local/lib/python3.5/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


infer time:  801.3098239898682
